# **Übung 3_2: Textklassifikator**

In [1]:
import matplotlib.pyplot as plt   # plotting
import numpy as np                # linear algebra
import os, os.path                # accessing directory structure
import pandas as pd               # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import sklearn
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
pd.show_versions(as_json=False)
np.__version__

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)



INSTALLED VERSIONS
------------------
commit           : 67a3d4241ab84419856b84fc3ebc9abcbe66c6b3
python           : 3.6.9.final.0
python-bits      : 64
OS               : Linux
OS-release       : 4.19.112+
Version          : #1 SMP Thu Jul 23 08:00:38 PDT 2020
machine          : x86_64
processor        : x86_64
byteorder        : little
LC_ALL           : None
LANG             : en_US.UTF-8
LOCALE           : en_US.UTF-8

pandas           : 1.1.4
numpy            : 1.18.5
pytz             : 2018.9
dateutil         : 2.8.1
pip              : 19.3.1
setuptools       : 50.3.2
Cython           : 0.29.21
pytest           : 3.6.4
hypothesis       : None
sphinx           : 1.8.5
blosc            : None
feather          : 0.4.1
xlsxwriter       : None
lxml.etree       : 4.2.6
html5lib         : 1.0.1
pymysql          : None
psycopg2         : 2.7.6.1 (dt dec pq3 ext lo64)
jinja2           : 2.11.2
IPython          : 5.5.0
pandas_datareader: 0.9.0
bs4              : 4.6.3
bottleneck       : 1

'1.18.5'

In [5]:
path_comp="/content/drive/MyDrive/Maschinelles Lernen/Übungen/20news-18828"
list = os.listdir(path_comp) # dir is directory path

In [ ]:
list

['soc.religion.christian',
 'sci.crypt',
 'sci.space',
 'rec.sport.hockey',
 'sci.electronics',
 'sci.med',
 'talk.politics.misc',
 'talk.politics.guns',
 'talk.religion.misc',
 'talk.politics.mideast',
 'comp.sys.ibm.pc.hardware',
 'alt.atheism',
 'rec.motorcycles',
 'rec.autos',
 'comp.windows.x',
 'rec.sport.baseball',
 'comp.os.ms-windows.misc',
 'misc.forsale',
 'comp.sys.mac.hardware',
 'comp.graphics']

b.)  Wir beschränken uns auf die vier Newsgroups alt.atheism, comp.graphics, sci.space
und talk.religion.misc. Lesen Sie alle Dateien aus diesen vier Verzeichnissen in eine
Array von Strings ein (d.h. ein File in einem String). Speichern Sie zusätzlich die Klassenzugehörigkeit
jedes Dokuments in einem Vektor ab (Kontrolle: Sie müssten jetzt 3387 Strings
im Speicher haben).

In [6]:
newsgroups = ["alt.atheism", "comp.graphics", "sci.space", "talk.religion.misc"]
data_raw = []
labels = []
for path, _, files in os.walk(path_comp):
    #print(files)
    #print(path)
    directory = path.split('/')[-1]
    #print(directory)
    if directory not in newsgroups:
        continue
    for file in files:
      with open(os.path.join(path, file), encoding="iso-8859-1") as myfile:
          data_raw.append(myfile.read())
          labels.append(directory)
    print(directory, " completed")

sci.space  completed
talk.religion.misc  completed
alt.atheism  completed
comp.graphics  completed


In [ ]:
len(data_raw)

3387

Die ersten 2 Zeilen (Header) aus jedem Dokument entfernen

In [8]:
def strip_header(text):
  _before, _blankline, after = text.partition('\n\n')
  return after

In [9]:
data = [strip_header(text) for text in data_raw]

In [ ]:
data

c.)  Im nächsten Schritt muss jeder String in Worte (Tokens) zerlegt werden, die durch Leerzeichen,
Kommas etc. voneinander getrennt sind.

In [10]:
data_regex = [re.compile(r"(?u)\b[a-zA-Z]+\b").findall(text.lower()) for text in data]

In [11]:
df=pd.DataFrame(data_regex)
regex_vector=np.asarray(df).flatten().T
len(regex_vector)

32281497

Drop NoneTypes

In [12]:
tokens = [] 
for val in regex_vector: 
    if val != None : 
        tokens.append(val)

In [13]:
len(tokens)

974120

Drop duplicates

In [14]:
tokens = np.unique(tokens)
len(tokens)

34588

Berechnen Sie für jeden Text einen
Merkmalsvektor, der für jedes Wort des Vokabulars seine Häufigkeit innerhalb des Texts
enthält.

In [15]:
def load_data(data_regex, different_tokens):
  n_tokens=len(different_tokens)
  n_entries=len(data_regex)
  data = np.zeros(shape=(n_entries, n_tokens), dtype=np.int)
  for j, article in enumerate(data_regex):

    for i, token in enumerate(different_tokens):
      count = article.count(token)

      if count > 0:
          data[j][i] = count
     
  data = np.asarray(data)
  #data = data + 1
    
  return data

In [16]:
matrix=load_data(data, tokens)

In [ ]:
# Tokens oben in Matrix einfügen
#Merkmalsmatrix=np.append(tokens, matrix)

# Labels als letzte Spalte in Matrix einfügen
#X=np.append(labels, Merkmalsmatrix, axis=1)

d. Verwenden Sie die ersten 60% der Daten als Trainingsdatensatz, die restlichen als Testdatensatz.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(matrix, labels, test_size=0.4)

In [ ]:
X_train

array([[536,   0,   0, ...,   0,   0,   0],
       [506,   0,   0, ...,   0,   0,   0],
       [ 16,   0,   0, ...,   0,   0,   0],
       ...,
       [163,   0,   0, ...,   0,   0,   0],
       [ 68,   0,   0, ...,   0,   0,   0],
       [ 48,   0,   0, ...,   0,   0,   0]])

In [ ]:
tokens.shape

(34588,)

In [ ]:
pij=pd.DataFrame(data=matrix, columns=tokens)

In [ ]:
pij

,a,aa,aaa,aaaa,aaaaagggghhhh,aaaahhh,aaai,aaauuugggghhhhh,aacs,aah,aai,aalborg,aam,aamrl,aams,aan,aangeboden,aangegeven,aantal,aao,aaoepp,aap,aaplay,aarhus,aario,aarnet,aaron,aaroundpluto,aarseth,aas,aasked,aatdb,aau,aavso,aawin,ab,aba,abad,abandon,abandoned,...,zorn,zoro,zoroaster,zoroastres,zoroastrian,zoroastrianism,zoroastrians,zr,zrdrwsht,zrtwsht,zsoft,zt,zubin,zubrin,zuck,zues,zug,zullen,zulu,zumder,zuni,zur,zurbrin,zurich,zurlo,zurvanism,zus,zvezdny,zvi,zvonko,zwaartepunten,zwak,zwakke,zware,zwarte,zwork,zyda,zyeh,zyklon,zyxel
0,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,35,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,40,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,14,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,39,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3382,16,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3383,40,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3384,46,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3385,16,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
newsgroups=np.unique(labels)
newsgroups

array(['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc'],
      dtype='<U18')

In [ ]:
# Gesamte Anzahl der Wörter einer Klasse (newsgroup)
Nij = {newsgroups[0] : 0, 
       newsgroups[1] : 0, 
       newsgroups[2] : 0, 
       newsgroups[3] : 0}

In [ ]:
for i in range(len(y_train)):
    Nij[y_train[i]] += X_train[i].sum()

In [ ]:
Nij

{'alt.atheism': 2978142,
 'comp.graphics': 2579544,
 'sci.space': 3105128,
 'talk.religion.misc': 2583760}

In [ ]:
n_tokens=len(tokens)
# Absolute Häufigkeit der einzelnen Wörter
nij = {newsgroups[0] : np.zeros(n_tokens, dtype=np.int), 
       newsgroups[1] : np.zeros(n_tokens, dtype=np.int), 
       newsgroups[2] : np.zeros(n_tokens, dtype=np.int), 
       newsgroups[3] : np.zeros(n_tokens, dtype=np.int)}

In [ ]:
for i in range(len(y_train)):
    nij[y_train[i]] += X_train[i]

In [ ]:
# Relative Häufigkeit der einzelnen Wörter innerhalb einer Klasse (newsgroup)
pij = {newsgroups[0] : np.zeros(n_tokens), 
       newsgroups[1] : np.zeros(n_tokens), 
       newsgroups[2] : np.zeros(n_tokens), 
       newsgroups[3] : np.zeros(n_tokens)}

In [ ]:
for group in newsgroups:
  # Berechnung der relativen Häufigkeit mit Laplace-Glättung
    pij[group] = (nij[group] + 1) / (Nij[group] + n_tokens)

In [ ]:
# Anteil der Worter einer Klasse im Vergleich zu allen Wörtern
pi = {newsgroups[0] : 0.0, 
      newsgroups[1] : 0.0, 
      newsgroups[2] : 0.0, 
      newsgroups[3] : 0.0}

In [ ]:
for i in range(len(y_train)):
    pi[y_train[i]] += 1

for group in newsgroups:
    pi[group] /= len(y_train)

Trainieren Sie damit einen multinomialen naiven Bayes-Klassifikator. Bestimmen
Sie den Anteil korrekter Klassifikationen auf Ihren Trainings- und Testdaten. Wie gut
generalisiert Ihr Klassifikator?

In [ ]:
corrects = np.full(shape=(len(y_test)), fill_value=False)
for i, lable in enumerate(y_test):
    max_p, max_group = -np.inf, None
    for group in newsgroups:
        p = np.sum(np.log(pij[group]) * X_test[i])
        
        if p > max_p:
            max_p = p
            max_group = group
            
    corrects[i] = (lable==max_group)
print("Korrekte Klassifikationen: {}; Total: {}; Accuracy: {}%".format(corrects.sum(), len(y_test), np.round(corrects.sum() / len(y_test), decimals=4)*100))

Korrekt klassifiziert: 1202; Total: 1355; Accuracy: 88.71%
